In [1]:
# pip install PyPDF2
# pip install fpdf

######## ALTERNATIVE with annoying watermark
# pip install aspose-words

In [2]:
import os
import glob
import json
import ast
from fpdf import FPDF
from openai import OpenAI


def load_api_key(config_file):
    with open(config_file) as f:
        config = json.load(f)
    client = OpenAI(api_key=config['api_key'])
    return client

def get_text_files(directory):
    list_of_files = glob.glob(os.path.join(directory, '*.txt'))
    latest_file, file_name = [""," "]
    if len(list_of_files) > 0:
        latest_file = max(list_of_files, key=os.path.getctime)
        full_name = os.path.basename(latest_file)
        file_name = os.path.splitext(full_name)
    return latest_file, file_name[0]

def read_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    return content

def output_directories(path,name,extension):
    if not os.path.exists(path):
        os.makedirs(path)
    n_files = ""
    # n_files = len(get_text_files(path))
    # n_files = n_files + 1
    # if n_files < 10:
    #     n_files = "0" + str(n_files)
    # else:
    #     n_files = str(n_files)
    nameFile = name + n_files + extension
    return(path + nameFile)

# from __future__ import print_function

def find_values(id, json_repr):
    results = []

    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results

directory = "./mealPlans/"
config_file = "./noSubmit/config.json"

client = load_api_key(config_file)


In [3]:

def get_completion(prompt, client, model="gpt-4-1106-preview", temperature=0, ):

    messages = [{"role": "system", "content": 'You are a useful and concise inventory manager in a grocery store'},
                {"role": "user", "content": prompt}]
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    # response = completion.model_dump_json(indent=2)
    return completion.choices[0].message.content # message["content"]



def get_completion_from_messages(messages, client, model="gpt-4", temperature=0):

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]

In [4]:
###################################
####### Ingredients  ############
###################################
# INPUT: Lists of Ingredients for each day in Danish.
# OUTPUT: English translation of the lists of Ingredients for each day.

# Setup new file of MealPlans

text_file, file_name = get_text_files(directory)

output_Ingredients_dir = "./outputIngredients/"
outputIngredientsFile = output_directories(output_Ingredients_dir, file_name + "_Ingredients_",".txt")

content = read_file(text_file)
prompt = f"""
Your task is to translate the  
lists of ingredients from Danish to English.\

Lists of ingredients: 
"""

content_prompt = f"""
```{content}```
"""
response = get_completion(prompt+content_prompt, client)

print(response)

#######################################
#######################################
# Saving results

f = open(outputIngredientsFile, "w")
f.write(response)
f.close()  
    

Here is the translation of the ingredients from Danish to English:

```2 regular onions
3 cloves of garlic
1 tbsp oil
1 cauliflower
1 can chickpeas
1 can chopped tomatoes
400 ml coconut milk, light
1 tbsp garam masala
1 tsp cumin
1 tsp paprika
1 tsp ginger
chili or cayenne pepper to taste
salt and pepper

2 naan breads
sour cream 5%

1 kg potatoes
0.5 kg carrots
400 g fish mince
oil for frying

15 g butter or flavor-neutral oil
2 tbsp wheat flour
3 dl semi-skimmed milk
1 dl potato water
1 pot parsley
2 tbsp lemon juice
salt and pepper

2 large bell peppers, red
1 onion
2 cloves of garlic
1 tbsp oil
1 can chopped tomatoes
2.5 dl cooking cream 4%
1 tbsp paprika
2 tbsp chili sauce, strong or to taste
salt and pepper
500 g dinner meatballs

300 g rice
350 g cauliflower rice

2 packs of wheat buns (8-12 pcs)
butter
6 pieces of mixed fruit
4 carrot sticks

1 portion flatbread (recipe)
1 tbsp oil
2 eggplants
2 regular onions
2 cloves of garlic
2 tsp oregano
1 tsp paprika
1 tsp rosemary
0.5 ts

In [5]:
###################################
####### ShoppingList  ############
###################################
# INPUT: Lists of Ingredients for each day in English.
# OUTPUT: Json object containing a table with the ingredients 
#   grouped by category and the quantities added up.

# Setup new file of ShoppingLists

text_file, file_name = get_text_files(output_Ingredients_dir)

output_ShoppingList_dir = "./outputShoppingList/"
outputShoppingListFile_txt = output_directories(output_ShoppingList_dir, file_name + "_ShoppingList_text",".txt")

content = read_file(text_file)
prompt = f"""

From the list of ingredients between <>, perform the following tasks: \
1. isolate the ingredients. \
2. group the ingredients by type and add up the quantities. \
3. make a shopping list grouping the type of ingredient using the following categories: \
- produce. \
- meat. \
- canned goods. \
- eggs, milk, yogurt, sour cream. \
- cheese. \
- cold cuts. \
- nuts. \
- spices. \
- 

If there are no ingredients in one category, do not list the category in the shopping list.

Output only the shopping list.

Lists of ingredients: 
"""

content_prompt = f"""
```{content}```
"""
response = get_completion(prompt+content_prompt, client)

print(response)

#######################################
#######################################
# Saving results

f = open(outputShoppingListFile_txt, "w")
f.write(response)
f.close()  
    

Shopping List:

Produce:
- Onions: 5 regular + 1 optional red onion
- Garlic: 14 cloves
- Cauliflower: 1 whole + 350 g cauliflower rice
- Potatoes: 1.5 kg
- Carrots: 0.5 kg + 4 sticks + 3 large
- Parsley: 1 pot
- Bell peppers, red: 2 large
- Mixed fruit: 6 pieces
- Eggplants: 2
- Cucumber: 1
- Iceberg lettuce or similar: 0.5
- Tomatoes: 4
- Broccoli: 1
- Parsnips or parsley roots: 600 g

Meat:
- Fish mince: 400 g
- Dinner meatballs: 500 g
- Whole chicken: 1

Canned Goods:
- Chickpeas: 1 can
- Chopped tomatoes: 2 cans

Eggs, Milk, Yogurt, Sour Cream:
- Semi-skimmed milk: 3 dl
- Sour cream 5%: 2.5 dl + 1 unspecified quantity
- Oma cooking cream 4%: 2 x 2.5 dl
- Cooking cream 4%: 2.5 dl

Cheese:
- (No items to list)

Cold Cuts:
- (No items to list)

Nuts:
- (No items to list)

Spices:
- Garam masala: 1 tbsp
- Cumin: 1 tsp
- Paprika: 2 tsp + 1 tbsp
- Ginger: 1 tsp
- Chili or cayenne pepper: to taste + 0.5 tsp
- Salt: multiple uses
- Pepper: multiple uses
- Oregano: 2 tsp
- Rosemary: 1 tsp


In [6]:

# from PyPDF2 import PdfMerger 

def create_pdf(input_file,outputName):
    # Create a new FPDF object
    pdf = FPDF()

    # Open the text file and read its contents
    with open(input_file, 'r') as f:
        text = f.read()

    # Add a new page to the PDF
    pdf.add_page()

    # Set the font and font size
    pdf.set_font('Arial', size=12)

    # Write the text to the PDF
    pdf.write(5, text)

    # Save the PDF
    pdf.output(outputName)

    # # If a template PDF is specified, merge it with the new PDF
    # merger = PdfMerger()
    # template_pdf = './template.pdf'
    # if template_pdf:
    #     merger.append(PdfMerger(open(template_pdf, 'rb')))
    #     merger.append(PdfMerger(open('output.pdf', 'rb')))
    #     merger.write('merged_output.pdf')

In [7]:


create_pdf(outputShoppingListFile_txt, file_name+'shoppingList_2.pdf')

# UNTIL HERE

# STOP

# STOP

# STOP

In [8]:
# ###################################
# ####### ShoppingList  ############
# ###################################
# # INPUT: Lists of Ingredients for each day in English.
# # OUTPUT: Json object containing a table with the ingredients 
# #   grouped by category and the quantities added up.

# # Setup new file of ShoppingLists

# text_file, file_name = get_text_files(output_Ingredients_dir)

# output_ShoppingList_dir = "./outputShoppingList/"
# outputShoppingListFile_json = output_directories(output_ShoppingList_dir, file_name + "_ShoppingList_",".json")

# content = read_file(text_file)
# prompt = f"""
# Your task is to create a table with all the ingredients grouped
# by category and quantity.\

# The output table must be in Json format.\

# Your response must only be a Json object and nothing else.\

# Lists of ingredients: 
# """

# content_prompt = f"""
# ```{content}```
# """
# response = get_completion(prompt+content_prompt, client)

# print(response)

# #######################################
# #######################################
# # Saving results

# f = open(outputShoppingListFile_json, "w")
# f.write(response)
# f.close()  
    

In [9]:
# ###################################
# ####### ShoppingList  ############
# ###################################
# # INPUT: Lists of Ingredients for each day in English.
# # OUTPUT: Json object containing a table with the ingredients 
# #   grouped by category and the quantities added up.

# # Setup new file of ShoppingLists

# text_file, file_name = get_text_files(output_Ingredients_dir)

# output_ShoppingList_dir = "./outputShoppingList/"
# outputShoppingListFile_md = output_directories(output_ShoppingList_dir, file_name + "_ShoppingList_markdown",".md")

# content = read_file(text_file)
# prompt = f"""
# Create a shopping list with all the ingredients grouped
# by aisle and all the ingredients quantities added up together.\

# Lists of ingredients: 
# """

# content_prompt = f"""
# ```{content}```
# """
# response = get_completion(prompt+content_prompt, client)

# print(response)

# #######################################
# #######################################
# # Saving results

# f = open(outputShoppingListFile_md, "w")
# f.write(response)
# f.close()  
    

In [10]:
# # This code example demonstrates how to convert a Markdoen file to a PDF document.
# import aspose.words as aw

# # Load an existing Markdown document
# doc = aw.Document(outputShoppingListFile)

# # Save the document as PDF
# doc.save(file_name + "_ShoppingList_markdown.pdf")

In [11]:
# import json
# outputShoppingListFile = './outputShoppingList/w52_plan_Ingredients_03_ShoppingList_03.json'
# json_Shopping = json.loads(outputShoppingListFile)


In [12]:
# import pandas as pd
# import matplotlib.pyplot as plt

# df = pd.read_json(outputShoppingListFile)
# df.head()